<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=419b7e637f87fc9f174fe76020fe33af73f20ca10f8e04aca329432271b63bef
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-09 13:08:32
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  11.04 L
Current:  1.40 C
-------------------
Today PnL: 47.74 K (0.34%)
Current PnL: -18.85 L (-12.68%)
CY Booked + Current PnL: -7.83 L (-5.26%)
-------------------
Total profit:  1.71 L
Total loss:  -20.56 L
-------------------
Total Booked + Current PnL: 19.15 L (15.74%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.86%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 78.50 L (55.93%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.95%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.97,58.0,H-LC,10.78,171853.0,15676.0,10414.0,0.32,10.04,6.06,16.71,22.0,1.51,1.33,29.23,X40N,BTT,HEALTHCARE
77,TTKPRESTIG,770.00,105.76,63.0,M-SC,5.88,88654.0,-12123.0,12217.0,0.67,-12.03,13.78,0.09,245.0,-0.99,0.69,16.47,OX40N,NTT,DURABLES
62,SBIN,863.00,-13.25,47.0,M-LC,4.28,214027.0,12548.0,14661.0,-0.14,6.23,6.85,13.51,60.0,0.86,1.65,19.61,XY25,NTT,BANKS
41,ITC,452.00,-37.92,51.0,H-LC,3.22,200490.0,352.0,20530.0,0.60,0.18,10.24,10.44,4.0,0.02,1.55,5.75,X40,NTT,FMCG
49,NATIONALUM,247.44,-44.49,68.0,H-MC,1.75,110447.0,9943.0,20698.0,-1.03,9.89,18.74,30.49,79.0,0.48,0.85,45.50,MH,ATH,METALS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,41.94,62.0,M-MC,8.03,224463.0,-499.0,168145.0,-0.84,-0.22,74.91,74.53,192.0,-0.00,1.74,31.96,XY24,BTT,STEEL
4,ANGELONE,3033.00,8.67,40.0,H-SC,6.49,192110.0,1104.0,56596.0,0.08,0.58,29.46,30.21,157.0,0.02,1.49,21.58,X40N,NTT,FINANCE
35,ICICIPRULI,790.00,-21.68,37.0,H-MC,2.08,134843.0,-945.0,43703.0,0.59,-0.70,32.41,31.48,107.0,-0.02,1.04,11.34,X40,ATH,INSURANCE
30,HAVELLS,2069.16,-0.37,60.0,H-MC,1.80,246948.0,-783.0,75838.0,0.24,-0.32,30.71,30.30,92.0,-0.01,1.91,12.93,X40,ATH,ELECTRICAL
34,ICICIGI,2260.25,-21.04,39.0,H-MC,3.71,135723.0,-410.0,31528.0,0.06,-0.30,23.23,22.86,91.0,-0.01,1.05,12.27,X40,ATH,INSURANCE


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HCLTECH,1943.91,-1.94,38.0,H-LC,10.17,223929.0,-17995.0,81264.0,1.66,-7.44,36.29,26.15,8.0,-0.22,1.73,5.82,X40,ATH,IT
43,JIOFIN,387.00,-2.28,39.0,H-LC,13.87,216055.0,-1351.0,54835.0,-0.79,-0.62,25.38,24.61,48.0,-0.02,1.67,53.86,XY24,BTT,FINANCE
83,VBL,671.64,-15.98,39.0,H-LC,6.66,302859.0,-12983.0,125656.0,0.14,-4.11,41.49,35.67,5.0,-0.10,2.34,9.17,X40N,ATH,FMCG
74,TCS,4998.00,-28.04,43.0,H-LC,14.19,286653.0,-59303.0,183171.0,1.01,-17.14,63.90,35.80,1.0,-0.32,2.22,1.55,X40,BTT,IT
80,UNITDSPR,1644.00,-14.66,44.0,H-LC,8.35,171626.0,-10133.0,45378.0,0.32,-5.57,26.44,19.39,86.0,-0.22,1.33,2.12,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-6.07,51.0,H-LC,1.60,233212.0,-12939.0,92142.0,1.84,-5.26,39.51,32.18,16.0,-0.14,1.80,28.89,X200,ATH,IT
50,NESTLEIND,1377.00,-9.11,57.0,H-LC,2.52,279770.0,14344.0,42441.0,0.67,5.40,15.17,21.40,11.0,0.34,2.16,13.13,XY25,NTT,FMCG
41,ITC,452.00,-37.92,51.0,H-LC,3.22,200490.0,352.0,20530.0,0.60,0.18,10.24,10.44,4.0,0.02,1.55,5.75,X40,NTT,FMCG
20,CIPLA,1795.00,-20.47,49.0,H-LC,5.81,212597.0,8097.0,33314.0,0.69,3.96,15.67,20.25,10.0,0.24,1.64,13.04,X40N,BTT,PHARMA
83,VBL,671.64,-15.98,39.0,H-LC,6.66,302859.0,-12983.0,125656.0,0.14,-4.11,41.49,35.67,5.0,-0.10,2.34,9.17,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,100.70,67.0,H-SC,12.85,127143.0,-15336.0,34863.0,6.61,-10.76,27.42,13.70,163.0,-0.44,0.98,50.06,OX40N,NTT,BANKS
81,VAIBHAVGBL,521.0,57.02,66.0,H-SC,11.02,104663.0,-47328.0,121451.0,5.62,-31.14,116.04,48.77,125.0,-0.39,0.81,21.82,XR,NTT,JEWELLERY
39,INFY,2275.0,-18.16,52.0,H-LC,10.56,319308.0,6420.0,165274.0,4.62,2.05,51.76,54.87,3.0,0.04,2.47,8.84,X40,BTT,IT
82,VALIANTORG,838.0,-337.06,65.0,H-SC,8.90,100880.0,-32406.0,117001.0,4.13,-24.31,115.98,63.47,139.0,-0.28,0.78,69.10,XR,NTT,CHEMICALS
75,TITAGARH,1548.0,-6.92,57.0,H-SC,7.87,152775.0,-39240.0,118126.0,3.31,-20.44,77.32,41.08,158.0,-0.33,1.18,29.46,XY24,NTT,ENGINEERING


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.0,3737.84,64.0,M-SC,7.79,119652.0,-27716.0,79533.0,-5.27,-18.81,66.47,35.16,236.0,-0.35,0.93,22.21,XY24,NTT,AUTO
22,COFFEEDAY,80.0,-37.63,61.0,L-SC,40.48,91008.0,-22541.0,62595.0,-2.89,-19.85,68.78,35.27,268.0,-0.36,0.70,122.74,XR,NTT,HOTELS
68,STARHEALTH,761.0,7.79,50.0,H-SC,3.72,160210.0,-40006.0,116040.0,-2.27,-19.98,72.43,37.97,144.0,-0.34,1.24,28.64,XY24,NTT,INSURANCE
67,SONACOMS,1006.0,-37.22,48.0,M-SC,12.31,78190.0,-22969.0,97863.0,-1.66,-22.71,125.16,74.03,202.0,-0.23,0.60,10.26,AR,BTT,AUTO
64,SHALBY,327.0,1075.44,74.0,M-SC,2.81,159847.0,-21296.0,66768.0,-1.57,-11.76,41.77,25.10,235.0,-0.32,1.24,28.10,XY24,NTT,HEALTHCARE


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,100.70,67.0,H-SC,12.85,127143.0,-15336.0,34863.0,6.61,-10.76,27.42,13.70,163.0,-0.44,0.98,50.06,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,105.76,63.0,M-SC,5.88,88654.0,-12123.0,12217.0,0.67,-12.03,13.78,0.09,245.0,-0.99,0.69,16.47,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.24,65.0,H-SC,2.55,225900.0,-43767.0,80104.0,3.16,-16.23,35.46,13.47,138.0,-0.55,1.75,14.26,XY24,NTT,PAINTS
37,INDIGOPNTS,1408.0,114.29,49.0,M-SC,3.24,140666.0,-33893.0,33929.0,-0.63,-19.42,24.12,0.02,221.0,-1.00,1.09,22.35,OX40N,NTT,PAINTS
19,CERA,9475.0,-20.05,45.0,H-SC,2.41,114165.0,-29638.0,56386.0,-0.66,-20.61,49.39,18.60,149.0,-0.53,0.88,25.98,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-28.04,43.0,H-LC,14.19,286653.0,-59303.0,183171.0,1.01,-17.14,63.90,35.80,1.0,-0.32,2.22,1.55,X40,BTT,IT
39,INFY,2275.00,-18.16,52.0,H-LC,10.56,319308.0,6420.0,165274.0,4.62,2.05,51.76,54.87,3.0,0.04,2.47,8.84,X40,BTT,IT
41,ITC,452.00,-37.92,51.0,H-LC,3.22,200490.0,352.0,20530.0,0.60,0.18,10.24,10.44,4.0,0.02,1.55,5.75,X40,NTT,FMCG
83,VBL,671.64,-15.98,39.0,H-LC,6.66,302859.0,-12983.0,125656.0,0.14,-4.11,41.49,35.67,5.0,-0.10,2.34,9.17,X40N,ATH,FMCG
1,ABB,7934.00,-41.03,46.0,H-LC,12.75,246413.0,-15206.0,134418.0,0.50,-5.81,54.55,45.57,7.0,-0.11,1.91,4.85,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BLUESTARCO,2326.38,9.17,53.0,H-MC,3.36,188510.0,23840.0,44130.0,0.05,14.48,23.41,41.28,89.0,0.54,1.46,23.62,X40N,ATH,AC
4,ANGELONE,3033.00,8.67,40.0,H-SC,6.49,192110.0,1104.0,56596.0,0.08,0.58,29.46,30.21,157.0,0.02,1.49,21.58,X40N,NTT,FINANCE
84,VOLTAS,1918.49,1.44,61.0,H-MC,3.47,212715.0,20973.0,75067.0,-0.08,10.94,35.29,50.08,99.0,0.28,1.64,18.48,XY25,ATH,AC
24,DABUR,735.00,-2.58,61.0,H-MC,5.15,212082.0,15054.0,74568.0,-0.44,7.64,35.16,45.49,102.0,0.20,1.64,21.68,XY24,BTT,FMCG
18,CAMS,5211.76,-4.94,52.0,H-SC,5.13,108088.0,6084.0,37842.0,0.39,5.96,35.01,43.06,122.0,0.16,0.84,24.10,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-44.49,68.0,H-MC,1.75,110447.0,9943.0,20698.0,-1.03,9.89,18.74,30.49,79.0,0.48,0.85,45.50,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-39.02,64.0,M-SC,3.10,103012.0,11514.0,67236.0,0.77,12.58,65.27,86.07,223.0,0.17,0.80,49.59,XR,NTT,DURABLES
36,INDIAMART,4850.92,-51.65,46.0,H-SC,12.07,135129.0,11793.0,121967.0,-0.97,9.56,90.26,108.45,119.0,0.10,1.04,34.05,AR,ATH,MISC
86,WIPRO,420.00,-12.43,50.0,M-LC,6.76,154194.0,3249.0,106209.0,2.55,2.15,68.88,72.51,53.0,0.03,1.19,8.30,XR,NTT,IT
12,BANKINDIA,190.00,-32.92,49.0,H-MC,6.58,173794.0,-6014.0,118423.0,-0.18,-3.34,68.14,62.52,88.0,-0.05,1.34,29.60,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6716.67,51.0,L-SC,6.19,74764.0,-19046.0,95661.0,1.21,-20.30,127.95,81.67,269.0,-0.20,0.58,44.78,XR,NTT,CERAMICS
3,ALKYLAMINE,4546.37,-6.52,40.0,H-SC,14.99,87079.0,-13884.0,108413.0,-0.51,-13.75,124.50,93.63,148.0,-0.13,0.67,29.70,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-37.63,61.0,L-SC,40.48,91008.0,-22541.0,62595.0,-2.89,-19.85,68.78,35.27,268.0,-0.36,0.70,122.74,XR,NTT,HOTELS
48,MASFIN,398.61,-16.44,59.0,H-SC,12.94,95130.0,-2850.0,24448.0,2.17,-2.91,25.70,22.05,152.0,-0.12,0.74,38.28,XR,ATH,FINANCE
82,VALIANTORG,838.00,-337.06,65.0,H-SC,8.90,100880.0,-32406.0,117001.0,4.13,-24.31,115.98,63.47,139.0,-0.28,0.78,69.10,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.0,-37.63,61.0,L-SC,40.48,91008.0,-22541.0,62595.0,-2.89,-19.85,68.78,35.27,268.0,-0.36,0.70,122.74,XR,NTT,HOTELS
8,ATULAUTO,844.0,3737.84,64.0,M-SC,7.79,119652.0,-27716.0,79533.0,-5.27,-18.81,66.47,35.16,236.0,-0.35,0.93,22.21,XY24,NTT,AUTO
60,SAMMAANCAP,326.0,-171.59,64.0,M-SC,4.59,82008.0,-20202.0,113589.0,-0.76,-19.77,138.51,91.37,208.0,-0.18,0.63,33.13,XY25,NTT,FINANCE
82,VALIANTORG,838.0,-337.06,65.0,H-SC,8.90,100880.0,-32406.0,117001.0,4.13,-24.31,115.98,63.47,139.0,-0.28,0.78,69.10,XR,NTT,CHEMICALS
55,RELAXO,1176.0,-31.84,60.0,H-SC,3.39,63226.0,-52312.0,87302.0,1.11,-45.28,138.08,30.28,136.0,-0.60,0.49,24.30,X40N,NTT,FOOTWEAR


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.31
1,25,44.52
2,50,75.04


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.89
LC    35.28
MC    23.82
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.75
X40      16.04
X40N     13.17
XY25     10.30
XR        9.56
AR        8.27
OX40N     7.76
X200      1.80
MH        1.78
X5K       1.45
SR        1.11
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.75
H-SC    24.98
H-MC    20.71
M-SC    14.23
M-LC     7.42
M-MC     2.78
L-SC     1.68
L-LC     1.11
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,12.77,-18.05,77.81
FMCG,12.35,-3.83,39.02
FINANCE,9.09,-19.00,61.79
BANKS,7.67,-14.85,63.61
PAINTS,6.23,-14.72,40.40
MISC,6.02,-18.07,85.75
ELECTRICAL,5.87,-8.43,68.19
AC,3.85,2.81,30.11
AUTO,3.54,-14.71,66.44


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2750507.0,22
AR,1199572.0,9
XR,1057932.0,13
X40,891804.0,10
X40N,573409.0,10
OX40N,454883.0,10
XY25,416809.0,7
SR,199300.0,2
X5K,129721.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2793942.0,28
M-SC,1406732.0,17
H-LC,1404477.0,15
H-MC,1165120.0,15
M-LC,417992.0,5
M-MC,318245.0,2
L-SC,243458.0,3
L-MC,60015.0,1
L-LC,40258.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,925115.0,6
M-SC,XY24,819546.0,7
H-SC,AR,664384.0,5
H-LC,X40,601383.0,5
H-SC,XR,498688.0,6
H-MC,XY24,465365.0,4
H-LC,AR,374750.0,2
M-MC,XY24,318245.0,2
H-SC,X40N,248975.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 53.0 seconds
